In [54]:
#Librairies import

import re
import numpy as np
import pandas as  pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spaCy for preprocessing
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import matplotlib.pyplot as plt
%matplotlib inline


import warnings
warnings.filterwarnings("ignore")
from wordcloud import WordCloud


In [2]:
#Data import

df_data = pd.read_csv(filepath_or_buffer=r"C:\Users\user\Downloads\Content (1).csv", sep=",")

C:\Users\user\AppData\Local\Temp\ipykernel_22204\2081215020.py:3: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255

In [5]:
 df_data.head(5)

,content_text,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 587,Unnamed: 588,Unnamed: 589,Unnamed: 590,Unnamed: 591,Unnamed: 592,Unnamed: 593,Unnamed: 594,Unnamed: 595,Unnamed: 596
0,Indian Constitution,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Economics is described due to the fact the tec...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,The concept of control is ubiquitous. While on...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Digital Mindset 101Appreciating the What?Clear...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Data science is an interdisciplinary field tha...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# NLTK Stop words

from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [27]:
# Convert to list 
data = df_data.content_text.values.tolist() 

# Remove Emails 
data = [re.sub( "\S*@\S*\s?", ' ', str(sent) ) for sent in data] 

# Remove new line characters 
data = [re.sub("\s+", ' ', str(sent)) for sent in data] 

# Remove distracting single quotes 
data = [re.sub("\'", "", str(sent)) for sent in data]

pprint(data[:2])


['Indian Constitution',
 'Economics is described due to the fact the technological know-how that '
 'offers with the assembly , distribution and intake of merchandise and '
 'offerings. Economics, which developed with inside the nineteenth century, '
 'has emerge as one of the maximum essential research of our time. From small '
 'stores to countries, economics performs an essential function with inside '
 'the green operation of both. Businesses cant thrive without making use of '
 'the standards of economics. Economics studies are huge and diverse. The '
 'nature and scope of economics relies upon at the interplay of financial '
 'marketers and consequently the mechanics of the financial system. Lets '
 'examine the character and scope of economics in depth.The nature of '
 'economicsThe nature of economics offers with the query of whether or not '
 'economics falls into the class of technological know-how or art. While '
 'numerous economists are arguing in desire of technological k

In [29]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
        
#deacc=True removes punctuations

data_words = list(sent_to_words(data))
print(data_words[:2])

[['indian', 'constitution'], ['economics', 'is', 'described', 'due', 'to', 'the', 'fact', 'the', 'technological', 'know', 'how', 'that', 'offers', 'with', 'the', 'assembly', 'distribution', 'and', 'intake', 'of', 'merchandise', 'and', 'offerings', 'economics', 'which', 'developed', 'with', 'inside', 'the', 'nineteenth', 'century', 'has', 'emerge', 'as', 'one', 'of', 'the', 'maximum', 'essential', 'research', 'of', 'our', 'time', 'from', 'small', 'stores', 'to', 'countries', 'economics', 'performs', 'an', 'essential', 'function', 'with', 'inside', 'the', 'green', 'operation', 'of', 'both', 'businesses', 'cant', 'thrive', 'without', 'making', 'use', 'of', 'the', 'standards', 'of', 'economics', 'economics', 'studies', 'are', 'huge', 'and', 'diverse', 'the', 'nature', 'and', 'scope', 'of', 'economics', 'relies', 'upon', 'at', 'the', 'interplay', 'of', 'financial', 'marketers', 'and', 'consequently', 'the', 'mechanics', 'of', 'the', 'financial', 'system', 'lets', 'examine', 'the', 'characte

In [33]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) 

# higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['indian', 'constitution']


In [34]:
# Define function for stopwords, bigrams, trigrams and lemmatization

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [39]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)


# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en

nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['constitution']]


In [40]:
# Create Dictionary 
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus 
texts = data_lemmatized

# Term Document Frequency 
corpus = [id2word.doc2bow(text) for text in texts]

# View 
print(corpus[:2])

[[(0, 1)], [(1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 3), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 2), (22, 1), (23, 2), (24, 1), (25, 2), (26, 2), (27, 2), (28, 3), (29, 1), (30, 2), (31, 1), (32, 2), (33, 1), (34, 6), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 3), (45, 1), (46, 2), (47, 2), (48, 2), (49, 1), (50, 1), (51, 2), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 14), (64, 1), (65, 1), (66, 2), (67, 1), (68, 4), (69, 2), (70, 2), (71, 1), (72, 1), (73, 2), (74, 2), (75, 1), (76, 2), (77, 2), (78, 1), (79, 1), (80, 1), (81, 1), (82, 2), (83, 1), (84, 1), (85, 4), (86, 1), (87, 12), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1), (94, 1), (95, 2), (96, 3), (97, 1), (98, 2), (99, 1), (100, 3), (101, 1), (102, 1), (103, 1), (104, 1), (105, 2), (106, 1), (107, 1), (108, 2), (109, 1), (110

In [43]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:2]]

[[('constitution', 1)],
 [('accord', 1),
  ('activity', 1),
  ('additionally', 1),
  ('advocate', 1),
  ('affair', 1),
  ('agency', 1),
  ('aggregate', 1),
  ('alternate', 1),
  ('amass', 1),
  ('analyze', 1),
  ('applicability', 1),
  ('argue', 1),
  ('art', 3),
  ('artit', 1),
  ('assembly', 1),
  ('assess', 1),
  ('assist', 1),
  ('attain', 1),
  ('attribute', 1),
  ('bank', 1),
  ('based_totally', 2),
  ('beneficial', 1),
  ('business', 2),
  ('capita', 1),
  ('causality', 2),
  ('character', 2),
  ('characteristic', 2),
  ('charge', 3),
  ('class', 1),
  ('clear', 2),
  ('commercial', 1),
  ('commodity', 2),
  ('component', 1),
  ('concept', 6),
  ('conclusion', 1),
  ('consequently', 1),
  ('consideration', 1),
  ('conversion', 1),
  ('cope', 1),
  ('corporation', 1),
  ('correlate', 1),
  ('correspond', 1),
  ('cost', 1),
  ('country', 3),
  ('course', 1),
  ('cycle', 2),
  ('deflation', 2),
  ('demand', 2),
  ('depression', 1),
  ('depth', 1),
  ('describe', 2),
  ('designate',

In [44]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [45]:
# Print the keyword of topics

pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.029*"cash" + 0.011*"liability" + 0.000*"media_planner" + 0.000*"radio" + '
  '0.000*"media" + 0.000*"ad" + 0.000*"tax_return" + '
  '0.000*"filing_requirement" + 0.000*"advertiser" + 0.000*"indicator"'),
 (1,
  '0.054*"art" + 0.042*"name" + 0.041*"head" + 0.035*"man" + 0.032*"carry" + '
  '0.027*"night" + 0.027*"stand" + 0.026*"close" + 0.024*"light" + '
  '0.024*"space"'),
 (2,
  '0.019*"content" + 0.017*"work" + 0.016*"time" + 0.014*"eye" + '
  '0.013*"negative" + 0.013*"find" + 0.013*"result" + 0.012*"know" + '
  '0.011*"also" + 0.011*"give"'),
 (3,
  '0.129*"creative" + 0.046*"energy" + 0.042*"solid" + 0.040*"liquid" + '
  '0.029*"constant" + 0.019*"temperature" + 0.019*"substance" + '
  '0.017*"integrate" + 0.013*"density" + 0.010*"producer"'),
 (4,
  '0.161*"file" + 0.026*"screen" + 0.001*"talent" + 0.000*"tax_return" + '
  '0.000*"filing_requirement" + 0.000*"tax_academy" + 0.000*"filing" + '
  '0.000*"taxes_generally" + 0.000*"calendar_year" + 0.000*"file_federal"'),


In [46]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))
# a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -14.574554410769391

Coherence Score:  0.4261819010955132


In [56]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.333801 -0.192964       1        1  27.829146
14     0.334865 -0.042197       2        1  16.444473
7      0.245236  0.025676       3        1  14.283265
15     0.243566  0.231978       4        1  13.360480
5      0.176990 -0.215071       5        1   7.358206
16     0.048197  0.011519       6        1   5.018632
1      0.080366 -0.037756       7        1   4.602960
9      0.159993  0.289986       8        1   3.909575
13    -0.131543 -0.012310       9        1   1.878645
19    -0.090595 -0.000275      10        1   1.588070
17    -0.118104 -0.004326      11        1   1.316244
3     -0.137000 -0.005504      12        1   0.787567
4     -0.144362 -0.006191      13        1   0.450091
12    -0.143970 -0.006134      14        1   0.353650
0     -0.142653 -0.006058      15        1   0.209769
11    -0.143982 -0.006145      16        1   0.196061
10    -0.143551 -0.006114      17        1   0.140376
6     -0.142954 -0.006069      18        1   0.126637
18    -0.141937 -0.006004      19        1   0.078646
8     -0.142363 -0.006040      20        1   0.067505, topic_info=             Term         Freq        Total Category  logprob  loglift
511        medium  2851.000000  2851.000000  Default  30.0000  30.0000
705       company  2415.000000  2415.000000  Default  29.0000  29.0000
2640        brand  1776.000000  1776.000000  Default  28.0000  28.0000
23       business  1066.000000  1066.000000  Default  27.0000  27.0000
92           good  1698.000000  1698.000000  Default  26.0000  26.0000
...           ...          ...          ...      ...      ...      ...
4930         love     0.008395     1.156339  Topic20  -9.6355   2.3753
11111   sentiment     0.008395     1.156002  Topic20  -9.6355   2.3756
13752       viral     0.008395     1.155863  Topic20  -9.6355   2.3757
143    prevalence     0.008395     1.155793  Topic20  -9.6355   2.3758
864         favor     0.008395     1.155927  Topic20  -9.6355   2.3756

[1065 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
564       4  0.988393    academic
437      12  0.411337  accelerate
570       1  0.996459      action
3903     10  0.921533    activate
2709     11  0.962845       adapt
...     ...       ...         ...
4087     17  0.560526      writer
1429      5  0.920270        year
1429      6  0.020450        year
1429      9  0.056807        year
2045      6  0.991440       young

[559 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 15, 8, 16, 6, 17, 2, 10, 14, 20, 18, 4, 5, 13, 1, 12, 11, 7, 19, 9])